### 1. Recolha e Preparação de dados

In [1]:
from pymongo import MongoClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F

client = MongoClient('mongodb://mongodb:27017/')
db = client.projeto
c_atrasos = db.atrasos
c_voos = db.voos

print(c_atrasos.find_one())
print(c_voos.find_one())

client.close()

{'_id': ObjectId('676ed62b177c65a442d95605'), 'year': 2023, 'month': 8, 'carrier': '9E', 'carrier_name': 'Endeavor Air Inc.', 'airport': 'ABE', 'airport_name': 'Allentown/Bethlehem/Easton, PA: Lehigh Valley International', 'arr_flights': 89.0, 'arr_del15': 13.0, 'carrier_ct': 2.25, 'weather_ct': 1.6, 'nas_ct': 3.16, 'security_ct': 0.0, 'late_aircraft_ct': 5.99, 'arr_cancelled': 2.0, 'arr_diverted': 1.0, 'arr_delay': 1375.0, 'carrier_delay': 71.0, 'weather_delay': 761.0, 'nas_delay': 118.0, 'security_delay': 0.0, 'late_aircraft_delay': 425.0}
{'_id': ObjectId('676ed668177c65a442dbf498'), '': 0, 'ItinID': 20181767585, 'MktID': 2018176758501, 'MktCoupons': 1, 'Quarter': 1, 'Origin': 'PHL', 'OriginWac': 23, 'Dest': 'LAX', 'DestWac': 91, 'Miles': 2402.0, 'ContiguousUSA': 2, 'NumTicketsOrdered': 1.0, 'AirlineCompany': 'AA', 'PricePerTicket': 672.87}


In [2]:
spark = SparkSession.builder \
    .appName("Voos e atrasos em 2018") \
    .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/projeto") \
    .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/projeto") \
    .getOrCreate()

df_atrasos = spark.read.format("mongo").option("collection", "atrasos").load()
df_voos = spark.read.format("mongo").option("collection", "voos").load()

In [3]:
print(df_atrasos.printSchema())
print(df_voos.printSchema())

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- arr_cancelled: double (nullable = true)
 |-- arr_del15: double (nullable = true)
 |-- arr_delay: double (nullable = true)
 |-- arr_diverted: double (nullable = true)
 |-- arr_flights: double (nullable = true)
 |-- carrier: string (nullable = true)
 |-- carrier_ct: double (nullable = true)
 |-- carrier_delay: double (nullable = true)
 |-- carrier_name: string (nullable = true)
 |-- late_aircraft_ct: double (nullable = true)
 |-- late_aircraft_delay: double (nullable = true)
 |-- month: integer (nullable = true)
 |-- nas_ct: double (nullable = true)
 |-- nas_delay: double (nullable = true)
 |-- security_ct: double (nullable = true)
 |-- security_delay: double (nullable = true)
 |-- weather_ct: double (nullable = true)
 |-- weather_delay: double (nullable = true)
 |-- year: integer (nullable = true)

None
root
 |-- AirlineC

In [4]:
df_atrasos = df_atrasos.drop("carrier","airport_name","FLIGHT_NUM","ORIGIN_SEQ_ID","DEST_SEQ_ID","DEP_TIME","ARR_TIME","WEATHER_DELAY")

df_atrasos = df_atrasos.dropna()

df_atrasos = df_atrasos.withColumn("index", monotonically_increasing_id())

df_atrasos = df_atrasos.dropDuplicates()

print(df_atrasos.printSchema())

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- arr_cancelled: double (nullable = true)
 |-- arr_del15: double (nullable = true)
 |-- arr_delay: double (nullable = true)
 |-- arr_diverted: double (nullable = true)
 |-- arr_flights: double (nullable = true)
 |-- carrier_ct: double (nullable = true)
 |-- carrier_delay: double (nullable = true)
 |-- carrier_name: string (nullable = true)
 |-- late_aircraft_ct: double (nullable = true)
 |-- late_aircraft_delay: double (nullable = true)
 |-- month: integer (nullable = true)
 |-- nas_ct: double (nullable = true)
 |-- nas_delay: double (nullable = true)
 |-- security_ct: double (nullable = true)
 |-- security_delay: double (nullable = true)
 |-- weather_ct: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- index: long (nullable = false)

None


In [5]:
print(df_atrasos.count())

171223


In [6]:
df_voos = df_voos.drop("ItinID","MktID","MktCoupons","ContiguousUSA","NumTicketsOrdered","AirlineCompany")
df_voos = df_voos.filter(df_voos["Quarter"] == 4)

df_voos = df_voos.dropna()

df_voos = df_voos.withColumn("index", monotonically_increasing_id())

df_voos = df_voos.dropDuplicates()

print(df_voos.printSchema())

root
 |-- Dest: string (nullable = true)
 |-- DestWac: integer (nullable = true)
 |-- Miles: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginWac: integer (nullable = true)
 |-- PricePerTicket: double (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- index: long (nullable = false)

None


In [7]:
print(df_voos.count())

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
           ^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <exception str() failed>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most 

Py4JError: An error occurred while calling z:py4j.reflection.TypeUtil.isInstanceOf